How to remove duplicate rows
</BR> Reference: https://www.youtube.com/watch?v=xw4a9qbOh-Q&list=PL50mYnndduIF868zbDUPMBpJpwJwd4NZh

In [21]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window

spark = SparkSession \
    .builder \
    .getOrCreate()

sc = spark.sparkContext

Read File

In [6]:
df = spark.read.csv('/home/phillipefs/spark_dev/pyspark-real-time-scenarios/data/duplicates.csv', header=True, inferSchema=True)
df.show()

+---+------+---------+-------------------+
| id|  name|      loc|       updated_date|
+---+------+---------+-------------------+
|  1|  ravi|bangalore|2021-01-01 00:00:00|
|  1|  ravi|  chennai|2022-02-02 00:00:00|
|  1|  ravi|Hyderabad|2022-06-10 00:00:00|
|  1|  ravi|Hyderabad|2022-06-10 00:00:00|
|  2|   Raj|bangalore|2021-01-01 00:00:00|
|  2|   Raj|  chennai|2022-02-02 00:00:00|
|  3|   Raj|Hyderabad|2022-06-10 00:00:00|
|  4|Prasad|bangalore|2021-01-01 00:00:00|
|  4|Prasad|bangalore|2021-01-01 00:00:00|
|  5|Mahesh|  chennai|2022-02-02 00:00:00|
|  4|Prasad|Hyderabad|2022-06-10 00:00:00|
|  4|Prasad|Hyderabad|2022-06-10 00:00:00|
+---+------+---------+-------------------+



Drop Duplicates Using drop_duplicates or distinct()

In [30]:
df.distinct()
df.drop_duplicates()

DataFrame[id: int, name: string, loc: string, updated_date: timestamp]

Drop Duplicates using specific column, keeps the most recent record.

In [20]:
df.orderBy(desc('updated_date')).drop_duplicates(['id']).show()

+---+------+---------+-------------------+
| id|  name|      loc|       updated_date|
+---+------+---------+-------------------+
|  1|  ravi|Hyderabad|2022-06-10 00:00:00|
|  2|   Raj|  chennai|2022-02-02 00:00:00|
|  3|   Raj|Hyderabad|2022-06-10 00:00:00|
|  4|Prasad|Hyderabad|2022-06-10 00:00:00|
|  5|Mahesh|  chennai|2022-02-02 00:00:00|
+---+------+---------+-------------------+



Window Function with row_number()

In [28]:
df.withColumn("rowid", row_number().over(Window.partitionBy("id").orderBy(col("updated_date").desc())))\
    .filter(col("rowid")==1)\
    .drop("rowid")\
    .show()

+---+------+---------+-------------------+
| id|  name|      loc|       updated_date|
+---+------+---------+-------------------+
|  1|  ravi|Hyderabad|2022-06-10 00:00:00|
|  2|   Raj|  chennai|2022-02-02 00:00:00|
|  3|   Raj|Hyderabad|2022-06-10 00:00:00|
|  4|Prasad|Hyderabad|2022-06-10 00:00:00|
|  5|Mahesh|  chennai|2022-02-02 00:00:00|
+---+------+---------+-------------------+

